In [1]:
import pandas as pd
df = pd.read_csv("catalog.csv")
df.head()

,Unnamed: 0,Year,Term,YearTerm,Subject,Number,Name,Description,Credit Hours,Section Info,Degree Attributes,Schedule Information
0,0,2026,Spring,2026-sp,AAS,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,NaN,"Social & Beh Sci - Soc Sci, and Cultural Studi...",NaN
1,1,2026,Spring,2026-sp,AAS,105,Introduction to Arab American Studies,Interdisciplinary introduction to the basic co...,3 hours.,NaN,Cultural Studies - US Minority course.,NaN
2,2,2026,Spring,2026-sp,AAS,120,Intro to Asian Am Pop Culture,Introductory understanding of the way U.S. pop...,3 hours.,NaN,Cultural Studies - US Minority course.,NaN
3,3,2026,Spring,2026-sp,AAS,199,Undergraduate Open Seminar,May be repeated to a maximum of 6 hours.,1 TO 5 hours.,NaN,NaN,NaN
4,4,2026,Spring,2026-sp,AAS,200,U.S. Race and Empire,Invites students to examine histories and narr...,3 hours.,Same as LLS 200.,"Humanities - Hist & Phil, and Cultural Studies...",NaN


In [ ]:
courseMap = {}

class Course:
  def __init__(self, id,  number, title):
    self.id = id
    self.title = title
    self.number = number
    self.prereqs = []
    self.fullPrereq = ""
    self.gened = False
    self.aka = []

In [56]:
import math
import re

for i, row in df.iterrows():

  id = f'{row["Subject"]} {row["Number"]}'
  description = row["Description"]
  title = row["Name"]
  course = Course(id, +row["Number"], title)
  courseMap[id] = course

  #description = row["Section Info"]
  if type(description).__name__ != "str": continue

  course.gened = (type(row["Degree Attributes"]).__name__ == "str")


  descriptionUpper = description.upper()

  if "PREREQ" in descriptionUpper:
    description = description.replace("Ph.D.", "PhD")
    description = description.replace("M.Eng.", "MEng")
    description = description.replace("course information for topics and prerequisites.", "")
    description = description.replace("class schedule for topics and prerequisites.", "")
    description = description.replace("Class Schedule for topics and prerequisites.", "")
    description = description.replace("department course information for topics and prerequisite.", "")
    
    description = description.replace("Jr. standing", "Junior standing")
    description = description.replace("M.A. standing", "MA standing")
    description = description.replace(". No prerequisites.", ".")
    description = description.replace("3.33 grade point", "B+ grade point")
    description = description.replace("3.3 GPA minimum", "3_3 GPA minimum")
    description = description.replace("grade-point average of 3.", "grade-point average of 3_")
    description = description.replace("Ed.D. students", "EdD students")
    description = description.replace("2.5 years of high school mathematics", "2_5 years of high school mathematics")
    description = description.replace("1.5 units of high school algebra", "1_5 units of high school algebra")

    if "PREREQ" not in description.upper():
      continue


    descriptionUpper = description.upper()


    prereqIndex = descriptionUpper.find("PREREQ")
    periodIndex = description.find(".", prereqIndex)
    if periodIndex <= 0: periodIndex = len(description)

    length = periodIndex - prereqIndex
    prereqString = description[prereqIndex:periodIndex]
    if prereqString.startswith("Prerequisite: "):
      prereqString = prereqString[14:]

    course.fullPrereq = prereqString

    #if "ONE OF" in prereqString.upper():
    #  print(prereqString)
    if " and " in prereqString and " or " in prereqString and ";" not in prereqString:
      og = prereqString
      if prereqString.count(" and ") == 1:
        prereqString = prereqString.replace(" and ", "; ")
        # print(f"{id} => {og} => {prereqString}")
      else:
        #print(f"{id} => {prereqString}")
        ...
    
    #if ";" in prereqString:
    #print(f"=== {id} ===")
    #print(prereqString)
    pieces = prereqString.split(";")
    for piece in pieces:
      if piece.upper().startswith("ONE OF"):
        subPieces = [piece]
      else:
        subPieces = piece.split(",")

      for s in subPieces:
        matches = re.findall(r'\b[A-Z]{2,4}\s*\d{3}\b', s)
        prereqList = list(set(matches))
        if len(prereqList) == 0:
          continue

        if " or " in s or s.upper().startswith("ONE OF"):
          course.prereqs.append(" | ".join(prereqList))
        else:
          for p in prereqList:
            if p != "":
              course.prereqs.append(p)

        ...


In [73]:
for id in courseMap:
  c = courseMap[id]
  if 200 <= c.number <= 299:
    if len(c.prereqs) > 1:
      print(f"{c.id}\t{c.prereqs}\t{c.fullPrereq}")
      ...

for id in courseMap:
  c = courseMap[id]
  if 200 <= c.number <= 299:
    if len(c.prereqs) == 1:
      singlePrereq = c.prereqs[0]
      if singlePrereq in courseMap:
        c2 = courseMap[singlePrereq]
        
        if len(c2.prereqs) != 0:
          print(f"{c.id}\t{c.prereqs} => {c2.prereqs}\t{c.fullPrereq} + [{c2.id}]: {c2.fullPrereq}")
          ...
      elif "|" not in singlePrereq:
        # print(singlePrereq)
        ...

print("====\t====\t====")

for id in courseMap:
  c = courseMap[id]
  if 200 <= c.number <= 299:
    if len(c.prereqs) == 1:
      singlePrereq = c.prereqs[0]
      if "|" in singlePrereq:
        print(f"{c.id} ??\t{c.prereqs}\t{c.fullPrereq}")
        ...


ACCY 201	['ACCY 201', 'ECON 103 | ECON 102', 'ECON 103 | ECON 102']	Prior to enrollment in ACCY 201, students must: 1) have completed either ECON 102 or ECON 103; AND 2) have completed or be concurrently enrolled in the remaining ECON 102 or ECON 103 course
ACCY 290	['ACCY 201', 'ACCY 202']	Open to Department of Accountancy students; completion of ACCY 201 and ACCY 202 (or equivalent); and consent of department
AFRO 290	['AFRO 101', 'HIST 276', 'HIST 172', 'PS 201']	AFRO 101, HIST 276, HIST 172, SOC, 225, or PS 201
AGED 250	['AGED 220', 'EDPR 203']	AGED 220; concurrent enrollment in EDPR 203
ANSC 221	['ANSC 100', 'CHEM 102']	ANSC 100, CHEM 102 and 103 or concurrent enrollment
ANSC 222	['ANSC 100', 'CHEM 102']	ANSC 100, CHEM 102 and 103 or concurrent enrollment
ANSC 223	['ANSC 100', 'ANSC 221']	ANSC 100 and ANSC 221
ANSC 224	['ANSC 100', 'ANSC 221']	ANSC 100, ANSC 221
ANTH 243	['ANTH 102', 'ANTH 143']	ANTH 102, ANTH 143, or an equivalent course in animal behavior; or consent of instruct

In [70]:
df[ (df.Subject == "CS") & (df.Number == 242) ].iloc[0].Description

'Intensive programming lab intended to strengthen skills in programming. Prerequisite: CS 241.'